# This notebook is dedicated to the data preprocessing of the Research Project
### MSC/DSA/134

In [7]:
# loading required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from globals.pandas_functions import *

In [5]:
# define datasets base path
data_base_path = "data/"

In [6]:
# import dataset
transaction_data_df = pd.read_csv(data_base_path + "processed/unified_transaction_data_option1.csv")

In [ ]:
dataset_dimension("")